<a href="https://colab.research.google.com/github/HeatherDriver/MathGraph/blob/main/05_NER_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install seqeval
! pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=e579ca36b87e2da684cf816e97f11964f10f090775919eec036f8b99abae2f29
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.1

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline, AutoConfig, DistilBertForTokenClassification, DistilBertModel, DistilBertConfig, DistilBertPreTrainedModel
from transformers import DataCollatorForTokenClassification, TrainingArguments, Trainer
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.tokenization_utils_base import BatchEncoding
from datasets import Dataset, DatasetDict
import torch
import torch.nn as nn
from google.colab import drive, userdata
import pickle
import random
import re
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.express as px
# from seqeval.metrics import classification_report, f1_score, precision_score, recall_score
from seqeval.metrics import classification_report
import evaluate
import pprint

In [3]:
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
metric = evaluate.load('seqeval')

In [6]:
# Define file read function
def read_pickle(dict_file):
  with open(dict_file, 'rb') as file:
    return pickle.load(file)

In [7]:
# Read in dictionary
all_data = read_pickle('all_data_matches.pkl')
all_data_amended = all_data.copy()

In [8]:
random_sample = random.sample(list(all_data_amended.items()), 3)

my_list = []
key, sub_dict = random_sample[0]
my_list.append(sub_dict['tokens'])
my_list.append(sub_dict['baseline_tags'])
my_list.append(sub_dict['input_ids'])
my_list.append(sub_dict['ner_tags'])
print(f"Key: {key}")
print(f"Text: {sub_dict['text']}")

pd.DataFrame(my_list, index=["Tokens", "Baseline_Tags", "Input_ids", "NER_Tags"])

Key: polynomial height
Text: Polynomial height refers to the maximum degree of the polynomial that represents a mathematical structure, often in the context of algebraic geometry or number theory. It provides a measure of the complexity of such structures by quantifying the size of the coefficients and the degree of the polynomial involved. This concept is useful in analyzing the behavior of polynomials in various applications, including optimization and cryptography.


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80
Tokens,[CLS],Pol,##yn,##omi,##al,height,refers,to,the,maximum,degree,of,the,polynomial,that,represents,a,mathematical,structure,",",often,in,the,context,of,algebraic,geometry,or,number,theory,.,It,provides,a,measure,of,the,complexity,of,such,structures,by,q,##uant,##ifying,the,size,of,the,coefficients,and,the,degree,of,the,polynomial,involved,.,This,concept,is,useful,in,analyzing,the,behavior,of,polynomial,##s,in,various,applications,",",including,optimization,and,cry,##pt,##ography,.,[SEP]
Baseline_Tags,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
Input_ids,101,17129,5730,18882,1348,3976,4431,1106,1103,4177,2178,1104,1103,19068,1115,5149,170,9988,2401,117,1510,1107,1103,5618,1104,19669,12053,1137,1295,2749,119,1135,2790,170,4929,1104,1103,12133,1104,1216,4413,1118,186,27280,8985,1103,2060,1104,1103,23795,1105,1103,2178,1104,1103,19068,2017,119,1188,3400,1110,5616,1107,23389,1103,4658,1104,19068,1116,1107,1672,4683,117,1259,25161,1105,5354,6451,9543,119,102
NER_Tags,O,B-CALCULUS-AND-ANALYSIS,O,O,O,E-CALCULUS-AND-ANALYSIS,O,O,O,O,O,O,O,S-CALCULUS-AND-ANALYSIS,O,O,O,O,O,O,O,O,O,O,O,B-GEOMETRY,S-GEOMETRY,O,B-NUMBER-THEORY,E-NUMBER-THEORY,O,O,O,O,S-CALCULUS-AND-ANALYSIS,O,O,S-FOUNDATIONS-OF-MATHEMATICS,O,O,O,O,S-NUMBER-THEORY,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,S-DISCRETE-MATHEMATICS,O,O,O,O,O,O,O,S-CALCULUS-AND-ANALYSIS,O,O,O,O,O,O,S-APPLIED-MATHEMATICS,O,O,O,O,O,O


In [ ]:
# Finetune definitions
# dict_file_name = 'train_definitions.pkl'
# train_definitions = read_pickle(dict_file_name)

# key, sub_dict = random_sample[0]
# # all_data_amended["Kobon Triangle"]['ner_tags'][68] = 'O'
# # all_data_amended["Kobon Triangle"]['ner_tags'][69] = 'O'

# # train_definitions.update({key: all_data_amended[key]})

# dict_file_name = 'train_definitions.pkl'
# with open(dict_file_name, 'wb') as file:
#   pickle.dump(train_definitions, file)

# # Train definitions has the hand-labelled definitions
# train_definitions = read_pickle(dict_file_name)
# train_definitions.keys()

dict_keys(["Simpson's Paradox", 'Gram-Schmidt Process', 'Cylindrical Parts', 'voter model', 'Skolem-Mahler-Lech Theorem', 'Many-To-One', 'Kobon Triangle'])

## Class distribution by main tag

In [9]:
# Function to count the tag definition per tag-list
def returns_count_per_class(ner_tag_list):
  my_dict = {}
  for tag in ner_tag_list:
    if tag != 'O':
      tag = '-'.join(tag.split('-')[1:])
    if tag not in my_dict:
      my_dict[tag] = 1
    else:
      my_dict[tag] += 1
  sorted_data = dict(sorted(my_dict.items(), key=lambda item: item[1], reverse=True))
  return sorted_data

In [10]:
# Function to summarise main tags
def returns_main_tags(all_data_amended):
  name, count_list, no_tag_list = [], [], []

  for key, sub_dict in all_data_amended.items():
    my_list, my_list_i = [],[]
    to_count = sub_dict['ner_tags']
    count = returns_count_per_class(to_count)

    for i, (k, v) in enumerate(count.items()):
      if k == 'O':
        continue
      else:
        my_list.append(k)
        my_list_i.append(key)
    try:
      count_list.append(my_list[0])
      name.append(my_list_i[0])
    except:
      no_tag_list.append(key)
      continue

  for_df = {'name': name, 'main_concept': count_list}
  df = pd.DataFrame(for_df)
  summary = df[['main_concept']].groupby('main_concept').value_counts().reset_index(drop=False)
  summary = summary.sort_values(by="count", ascending=False).reset_index(drop=True)
  return summary, df, no_tag_list

In [11]:
summary, label_df, no_tag_list = returns_main_tags(all_data_amended)

for tag in no_tag_list:
  try:
    del all_data_amended[tag]
  except:
    continue

assert len(all_data_amended) == label_df.shape[0], 'Error'

In [12]:
fig = px.bar(summary, x='main_concept', y='count', hover_data=['main_concept', 'count'], text='count', color='count', height=500, color_continuous_scale='Viridis', title="Distribution by primary classification label")
fig.update_coloraxes(showscale=False)
fig.show()

### Merging labels together

In [13]:
# Condense matrix classifications to single classification = "MATRICES"

b_tags = ["B-MATRIX-DECOMPOSITION", "B-MATRIX-EIGENVALUES", "B-MATRIX-GROUPS", "B-MATRIX-INVERSION", "B-MATRIX-NORMS", "B-MATRIX-OPERATIONS", "B-MATRIX-PROPERTIES", "B-MATRIX-TYPES", "B-INTEGER-MATRICES"]
i_tags = ["I-MATRIX-DECOMPOSITION","I-MATRIX-EIGENVALUES","I-MATRIX-GROUPS","I-MATRIX-INVERSION","I-MATRIX-NORMS","I-MATRIX-OPERATIONS","I-MATRIX-PROPERTIES","I-MATRIX-TYPES", "I-INTEGER-MATRICES"]
e_tags = ["E-MATRIX-DECOMPOSITION","E-MATRIX-EIGENVALUES","E-MATRIX-GROUPS","E-MATRIX-INVERSION","E-MATRIX-NORMS","E-MATRIX-OPERATIONS","E-MATRIX-PROPERTIES","E-MATRIX-TYPES", "E-INTEGER-MATRICES"]
s_tags = ["S-MATRIX-DECOMPOSITION","S-MATRIX-EIGENVALUES","S-MATRIX-GROUPS","S-MATRIX-INVERSION","S-MATRIX-NORMS","S-MATRIX-OPERATIONS","S-MATRIX-PROPERTIES","S-MATRIX-TYPES", "S-INTEGER-MATRICES"]
lie_b_tags = ["B-LIE-GROUPS", "B-LIE-THEORY"]
lie_i_tags = ["I-LIE-GROUPS", "I-LIE-THEORY"]
lie_e_tags = ["E-LIE-GROUPS", "E-LIE-THEORY"]
lie_s_tags = ["S-LIE-GROUPS", "S-LIE-THEORY"]

for i, (key, sub_dict) in enumerate(all_data_amended.items()):
  ner_tags = sub_dict['ner_tags']
  for ner_tag in ner_tags:
    if ner_tag in b_tags:
      ner_tags[ner_tags.index(ner_tag)] = "B-MATRICES"
    elif ner_tag in i_tags:
      ner_tags[ner_tags.index(ner_tag)] = "I-MATRICES"
    elif ner_tag in e_tags:
      ner_tags[ner_tags.index(ner_tag)] = "E-MATRICES"
    elif ner_tag in s_tags:
      ner_tags[ner_tags.index(ner_tag)] = "S-MATRICES"
    elif ner_tag in lie_b_tags:
      ner_tags[ner_tags.index(ner_tag)] = "B-LIE-ALGEBRA"
    elif ner_tag in lie_i_tags:
      ner_tags[ner_tags.index(ner_tag)] = "I-LIE-ALGEBRA"
    elif ner_tag in lie_e_tags:
      ner_tags[ner_tags.index(ner_tag)] = "E-LIE-ALGEBRA"
    elif ner_tag in lie_s_tags:
      ner_tags[ner_tags.index(ner_tag)] = "S-LIE-ALGEBRA"
  all_data_amended[key]['ner_tags'] = ner_tags

summary, label_df, no_tag_list = returns_main_tags(all_data_amended)
label_df = label_df.reset_index(drop=False)
fig = px.bar(summary, x='main_concept', y='count', hover_data=['main_concept', 'count'], text='count', color='count', height=500, color_continuous_scale='Viridis', title="Distribution by primary classification label")
fig.update_coloraxes(showscale=False)
fig.show()

In [14]:
# Check order of label_df same as all_data_amended (NB for indices for train-test split)
for i, key in enumerate(list(all_data_amended.keys())):
  assert label_df.iloc[i]['name'] == key, 'Error'

label_df.head()

,index,name,main_concept
0,0,ludwig's inversion formula,CALCULUS-AND-ANALYSIS
1,1,quotient,NUMBER-THEORY
2,2,survivorship curve,APPLIED-MATHEMATICS
3,3,inadmissible,DISCRETE-MATHEMATICS
4,4,Natural Logarithm of 2,DISCRETE-MATHEMATICS


## Train-test stratified split

In [15]:
X_train_indices, X_test_indices, y_train_indices, y_test_indices = train_test_split(label_df['index'].to_numpy(), label_df['main_concept'].to_numpy(),
                                                                                    test_size=0.3, random_state=42, stratify=label_df['main_concept'].to_numpy())

# Repeat to get validation sub-sample of Train
X_train_indices, X_valid_indices, y_train_indices, y_valid_indices = train_test_split(X_train_indices, y_train_indices, test_size=0.3, random_state=42, stratify=y_train_indices)

In [16]:
# Check the resulting distributions
print("Class distribution in original dataset:")
print(label_df['main_concept'].value_counts(normalize=True))

print("\nClass distribution in train dataset:")
print(pd.Series(y_train_indices).value_counts(normalize=True))

print("\nClass distribution in validation dataset:")
print(pd.Series(y_valid_indices).value_counts(normalize=True))

Class distribution in original dataset:
main_concept
CALCULUS-AND-ANALYSIS          0.215533
NUMBER-THEORY                  0.179963
TOPOLOGY                       0.108640
ALGEBRA                        0.100919
GEOMETRY                       0.092188
DISCRETE-MATHEMATICS           0.073621
PROBABILITY-AND-STATISTICS     0.069301
RECREATIONAL-MATHEMATICS       0.037408
APPLIED-MATHEMATICS            0.033915
MATRICES                       0.033272
FOUNDATIONS-OF-MATHEMATICS     0.031618
ORG                            0.008272
LOC                            0.005239
LINEAR-ALGEBRA                 0.004871
DETERMINANTS                   0.001838
LINEAR-SYSTEMS-OF-EQUATIONS    0.001746
LIE-ALGEBRA                    0.001654
Name: proportion, dtype: float64

Class distribution in train dataset:
CALCULUS-AND-ANALYSIS          0.215532
NUMBER-THEORY                  0.180079
TOPOLOGY                       0.108610
ALGEBRA                        0.100919
GEOMETRY                       0.092

## Developing the Model Parameters

In [17]:
# Massive class imbalance between labels - will modify the loss function to perform a weighted loss based on frequency of classes.
# The weights will just be 1 minus the proportion for the main label.
# Eg for  CALCULUS-AND-ANALYSIS = 1 - 0.215533 = 0.784467

_total_count_dictionary = dict()

for i, (key, sub_dict) in enumerate(all_data_amended.items()):
  my_dict = dict()
  my_list, my_list_i = [],[]
  to_count = sub_dict['ner_tags']
  for tag in to_count:
    if tag not in my_dict:
      my_dict[tag] = 1
    else:
      my_dict[tag] += 1
  for k, v in my_dict.items():
    if k not in _total_count_dictionary:
      _total_count_dictionary[k] = v
    else:
      _total_count_dictionary[k] += v

total_count_dictionary = dict(sorted(_total_count_dictionary.items(), key=lambda item: item[1], reverse=True))

total_count = sum(total_count_dictionary.values())
proportions = {k : v/total_count for k, v in total_count_dictionary.items()}

assert np.round(sum(proportions.values()),0) == 1, 'Error'

weights  = {k : 1 - v for k, v in proportions.items()}
class_weights = torch.tensor([weights[cls] for cls in sorted(proportions.keys())], dtype=torch.float32)

loss_fct = nn.CrossEntropyLoss(weight=class_weights)

In [18]:
# Load DistilBERT

tag_checkpoint = "dslim/distilbert-NER"
tag_tokenizer = AutoTokenizer.from_pretrained(tag_checkpoint, do_lower_case=False)
tag_model = AutoModelForTokenClassification.from_pretrained(tag_checkpoint)

tags = pipeline("ner", model=tag_model, tokenizer=tag_tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/926 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

Device set to use cuda:0


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_collator = DataCollatorForTokenClassification(tokenizer=tag_tokenizer, padding=True, return_tensors="pt")

In [20]:
# Need mapping of NER tags to their indices for model to use

tag_list = []
for key, sub_dict in all_data_amended.items():
  ner_tags = sub_dict['ner_tags']
  for tag in ner_tags:
    if tag not in tag_list:
      tag_list.append(tag)

tag_list = sorted(tag_list, key=lambda x: x.split('-', 1)[1] if '-' in x else '')

index2tag = {idx:tag for idx, tag in enumerate(tag_list)} # This is just a nonsignificant arbitrary mapping of the label to a number for training the model
tag2index = {tag:idx for idx, tag in enumerate(tag_list)} # To lookup indices from tags

index2tag[-0] = 'O'

tag2index['O'] = 0 # O-tags set to -100 so that ignored and subwords are not trained upon
index2tag

{0: 'O',
 1: 'B-ALGEBRA',
 2: 'E-ALGEBRA',
 3: 'S-ALGEBRA',
 4: 'I-ALGEBRA',
 5: 'B-APPLIED-MATHEMATICS',
 6: 'E-APPLIED-MATHEMATICS',
 7: 'S-APPLIED-MATHEMATICS',
 8: 'I-APPLIED-MATHEMATICS',
 9: 'B-CALCULUS-AND-ANALYSIS',
 10: 'I-CALCULUS-AND-ANALYSIS',
 11: 'E-CALCULUS-AND-ANALYSIS',
 12: 'S-CALCULUS-AND-ANALYSIS',
 13: 'S-DETERMINANTS',
 14: 'B-DETERMINANTS',
 15: 'E-DETERMINANTS',
 16: 'I-DETERMINANTS',
 17: 'S-DISCRETE-MATHEMATICS',
 18: 'B-DISCRETE-MATHEMATICS',
 19: 'I-DISCRETE-MATHEMATICS',
 20: 'E-DISCRETE-MATHEMATICS',
 21: 'S-FOUNDATIONS-OF-MATHEMATICS',
 22: 'B-FOUNDATIONS-OF-MATHEMATICS',
 23: 'E-FOUNDATIONS-OF-MATHEMATICS',
 24: 'I-FOUNDATIONS-OF-MATHEMATICS',
 25: 'S-GEOMETRY',
 26: 'B-GEOMETRY',
 27: 'E-GEOMETRY',
 28: 'I-GEOMETRY',
 29: 'B-LIE-ALGEBRA',
 30: 'E-LIE-ALGEBRA',
 31: 'S-LIE-ALGEBRA',
 32: 'I-LIE-ALGEBRA',
 33: 'B-LINEAR-ALGEBRA',
 34: 'E-LINEAR-ALGEBRA',
 35: 'I-LINEAR-ALGEBRA',
 36: 'S-LINEAR-ALGEBRA',
 37: 'S-LINEAR-INDEPENDENCE',
 38: 'B-LINEAR-SYSTEMS

In [21]:
# Replace ner_indices (labels) in all_data_amended with the index from index2tag

for key, sub_dict in all_data_amended.items():
  ner_tags = sub_dict['ner_tags']
  my_list = []
  for tag in ner_tags:
    idx = tag2index[tag]
    my_list.append(idx)
  all_data_amended[key]['labels'] = my_list

In [22]:
# Custom model based off of pretrained DistilBERT

class DistilBertForTokenClassification(DistilBertPreTrainedModel):

  def __init__(self, config):
    super().__init__(config)
    self.num_labels = config.num_labels

    # Model body
    self.distilbert = DistilBertModel(config)

    # Classification head
    self.dropout = nn.Dropout(config.dropout)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    # Initialise weights
    self.init_weights()

  def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
    # Remove num_items_in_batch from kwargs if present
    kwargs.pop('num_items_in_batch', None)

    outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask, **kwargs)
    sequence_output = self.dropout(outputs[0])
    logits = self.classifier(sequence_output)

    # Loss calc
    loss = None
    if labels is not None:
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

    final_output = TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)
    return final_output

In [23]:
# Update the existing config to the settings for custom DistilBERT
config = AutoConfig.from_pretrained(tag_checkpoint, num_labels=len(index2tag), label2id=tag2index, id2label=index2tag,)
config.label2id = tag2index
config.id2label = index2tag
config.num_labels = len(index2tag)
config.max_position_embeddings = 512

# Initialise custom DistilBERT using these changed configurations
tag_model_custom = DistilBertForTokenClassification(config)

In [24]:
# Load the pretrained state_dict
pretrained_model = AutoModelForTokenClassification.from_pretrained(tag_checkpoint)

# Use pretrained state_dict in tag_model_custom
tag_model_custom.distilbert.load_state_dict(pretrained_model.distilbert.state_dict(), strict=False)

tag_model_custom.classifier = nn.Linear(config.hidden_size, config.num_labels)
tag_model_custom.init_weights()

tag_model_custom.to(device)

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
   

## Checking untrained prediction ability

In [25]:
data_dict = dict()
for key, sub_dict in all_data_amended.items():
  my_dict = {k : [v] for k, v in sub_dict.items()}
  data_dict[key] = BatchEncoding(my_dict)

# Convert dictionary to BatchEncoding
batch_encoded_data = BatchEncoding(data_dict)

assert len(batch_encoded_data.keys()) == len(all_data_amended.keys()), 'Error'

In [26]:
# Check key ordering the same between all_data_amended and batch_encoded_data

my_list = list(batch_encoded_data.keys())
for i, key in enumerate(list(all_data_amended.keys())):
  assert my_list[i] == key, 'Error'

In [27]:
def pads_to_max_dimensions(sub_dict):
  # Model needs labels, input_ids and attention_mask to be the maximum size of 512, function pads remainder out to this size
  if isinstance(sub_dict['input_ids'], list):
    input_ids_tensor = torch.tensor(sub_dict['input_ids'], dtype=torch.long)  # Convert list to tensor
  else:
    input_ids_tensor = sub_dict['input_ids'].clone().detach()

  # Convert attention_mask
  if isinstance(sub_dict['attention_mask'], list):
    attention_mask_tensor = torch.tensor(sub_dict['attention_mask'], dtype=torch.long)  # Convert list to tensor
  else:
    attention_mask_tensor = sub_dict['attention_mask'].clone().detach()

  # Convert labels
  if isinstance(sub_dict['labels'], list):
    labels_mask_tensor = torch.tensor(sub_dict['labels'], dtype=torch.long)  # Convert list to tensor
  else:
    labels_mask_tensor = sub_dict['labels'].clone().detach()

  # Padding lengths
  max_len = config.max_position_embeddings
  input_padding_len = max_len - input_ids_tensor.shape[1]
  attention_padding_len = max_len - attention_mask_tensor.shape[1] #config.max_position_embeddings - attention_mask_tensor.shape[0]
  labels_padding_len = max_len - labels_mask_tensor.shape[1]

  # Padded out to the number of dimensions (512)
  dim_input_ids = torch.nn.functional.pad(input_ids_tensor, (0, input_padding_len), value=0)
  dim_attention_mask = torch.nn.functional.pad(attention_mask_tensor, (0, attention_padding_len), value=0)
  dim_labels = torch.nn.functional.pad(labels_mask_tensor, (0, labels_padding_len), value=-100)

  sub_dict['input_ids'] = dim_input_ids
  sub_dict['attention_mask'] = dim_attention_mask
  sub_dict['labels'] = dim_labels

  return sub_dict

In [28]:
# Implementing above fn
for key, sub_dict in batch_encoded_data.items():
  batch_encoded_data[key] = pads_to_max_dimensions(sub_dict)

## Defining Model functions

In [29]:
idx2key = {idx: key for idx, key in enumerate(batch_encoded_data.keys())}

# Function to create Hugging Face dataset split
def create_dataset_split(indices, batch_encoded_data):

  input_ids = [batch_encoded_data[idx2key[i]]['input_ids'] for i in indices]
  attention_mask = [batch_encoded_data[idx2key[i]]['attention_mask'] for i in indices]
  labels = [batch_encoded_data[idx2key[i]]['labels'] for i in indices]

  dataset_dict = {
      'input_ids': input_ids,
      'attention_mask': attention_mask,
      'labels': labels
  }
  return Dataset.from_dict(dataset_dict)

# Initialises custom model
def model_init():
  return tag_model_custom.to(device)

In [30]:
# Create train, validation and test sets

train_dataset = create_dataset_split(X_train_indices, batch_encoded_data)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

validation_dataset = create_dataset_split(X_valid_indices, batch_encoded_data)
validation_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

test_dataset = create_dataset_split(X_test_indices, batch_encoded_data)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Combine into a DatasetDict
dataset = DatasetDict({'train': train_dataset, 'validation': validation_dataset, 'test': test_dataset})

## Test Model function on single example

In [31]:
sample = train_dataset[0]
_inputs = {
    'input_ids': sample['input_ids'].clone().detach(),
    'attention_mask': sample['attention_mask'].clone().detach()}
inputs = {
    'input_ids': _inputs['input_ids'].to(device),
    'attention_mask': _inputs['attention_mask'].to(device)}

tag_model_custom.eval()

with torch.no_grad():
  outputs = tag_model_custom.forward(**inputs).logits

y_actual = sample['labels'].to(device)

In [32]:
prediction_idx = torch.argmax(outputs, dim=-1)
prediction_batch_size, prediction_dim = prediction_idx.shape
actual_dim = 512
actual_idx = 1
actual, predicted, _actual, _preds = [], [], [], []
for i in range(actual_dim):
  actual_idx = y_actual[0][i].item()
  pred_idx = prediction_idx[0][i].item()
  if pred_idx == 0:
    pred_idx = -100
  if actual_idx != -100 and actual_idx > 0:
    _preds.append(index2tag[pred_idx])
    _actual.append(index2tag[actual_idx])
  if actual_idx == 0:
    break
  predicted.append(_preds)
  actual.append(_actual)

In [33]:
# Display in df: across all 512 dimensions as a double-check
input_ids = inputs['input_ids'][0].cpu().numpy() #X
tokens = tag_tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].cpu().numpy()) #X in human sub-token form
prediction_idx = torch.argmax(outputs, dim=-1)
untrained_model_id = prediction_idx[0].cpu().numpy() #Tag predictions in numeric form
y_pred = [index2tag.get(p, 'O') for p in untrained_model_id] #Tag predictions in human word form
attn_mask = inputs['attention_mask'][0].cpu().numpy()
actual_idx = sample['labels'].squeeze().numpy() #Actual tag index
actual_ner = [index2tag[i] for i in actual_idx if i!= -100] #Actual tags

pd.DataFrame([input_ids, tokens, untrained_model_id, y_pred, attn_mask, actual_ner, actual_idx],
             index=["Input_ids", "Tokens", "Untrained_ID","Untrained_Tag_Prediction", "Attention_Mask", "Actual_NER_Tags", "Actual_NER_idx"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
Input_ids,101,138,6595,14235,1110,170,2079,1104,4718,1215,1106,10301,1126,3655,1416,17816,117,7713,1103,2178,1104,13265,1213,1103,10301,119,1135,2790,1126,14235,1439,1134,1195,5363,1103,2276,17816,2860,1106,4277,117,1359,1113,170,3468,6595,1634,117,3417,4448,1112,170,6556,119,1188,6806,6618,1107,23529,1103,13218,1104,6876,10777,1105,1543,1107,16792,1116,1164,2610,6623,119,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Tokens,[CLS],A,confidence,interval,is,a,range,of,values,used,to,estimate,an,unknown,population,parameter,",",indicating,the,degree,of,uncertainty,around,the,estimate,.,It,provides,an,interval,within,which,we,expect,the,true,parameter,value,to,lie,",",based,on,a,chosen,confidence,level,",",typically,expressed,as,a,percentage,.,This,tool,helps,in,assessing,the,precision,of,sample,estimates,and,making,in,##ference,##s,about,larger,populations,.,[SEP],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],

In [34]:
print(classification_report(actual, predicted))

ValueError: max() arg is an empty sequence

## Model Training

In [35]:
# Fixing input_ids shape in the dataset

def fix_shape(example):
  example['input_ids'] = example['input_ids'].squeeze(0).tolist()
  example['attention_mask'] = example['attention_mask'].squeeze(0).tolist()
  example['labels'] = example['labels'].squeeze(0).tolist()
  return example

dataset['train'] = dataset['train'].map(fix_shape)
dataset['validation'] = dataset['validation'].map(fix_shape)
dataset['test'] = dataset['test'].map(fix_shape)

Map:   0%|          | 0/5331 [00:00<?, ? examples/s]

Map:   0%|          | 0/2285 [00:00<?, ? examples/s]

Map:   0%|          | 0/3264 [00:00<?, ? examples/s]

In [36]:
# Update function so that it runs on Batch
def returns_actual_and_predictions(outputs, labels):
  # Convert outputs to a PyTorch tensor if it's a NumPy array
  outputs = torch.from_numpy(outputs) if isinstance(outputs, np.ndarray) else outputs

  prediction_idx = torch.argmax(outputs, dim=-1)
  prediction_batch_size, prediction_dim = prediction_idx.shape
  actual, predicted = [], []

  for i in range(prediction_batch_size):
    _labels, _preds = [], []
    for j in range(prediction_dim):
      label = labels[i][j].item()
      # ignore label of -100
      if label != -100:
        _labels.append(index2tag[label])
        _preds.append(index2tag[prediction_idx[i][j].item()])
    actual.append(_labels)
    predicted.append(_preds)
  return actual, predicted

def compute_metrics(eval_pred):
  actual_tags, predicted_tags = returns_actual_and_predictions(eval_pred.predictions, eval_pred.label_ids)
  metric_result_dict = metric.compute(predictions=predicted_tags, references=actual_tags)
  f1_score = metric_result_dict['overall_f1']
  return {'f1_score': f1_score}

In [37]:
num_epochs = 10
batch_size = 5
learning_rate = 3e-5
# logging_steps = len(dataset["train"]) // batch_size
model_name = f"distilbert-NER-Math-finetuned"

model_arguments = TrainingArguments(
    output_dir=model_name,
    log_level="error",
    logging_strategy="steps",
    logging_steps=50,
    weight_decay=0.01,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    eval_strategy="epoch",
    disable_tqdm=False,
    save_steps=1000000,
    remove_unused_columns=False,
    push_to_hub=True,
    no_cuda=False
)

data_collator = DataCollatorForTokenClassification(
    tokenizer=tag_tokenizer,
    return_tensors="pt",
    padding='max_length',
    label_pad_token_id=-100  # Padding token for labels
)

In [38]:
trainer = Trainer(
    model_init=model_init, args=model_arguments, data_collator=data_collator, compute_metrics=compute_metrics,
    tokenizer=tag_tokenizer, train_dataset=dataset['train'], eval_dataset=dataset['validation']
    )

<ipython-input-38-ceff5e08bcc7>:1: FutureWarning:

`tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.



In [39]:
torch.cuda.empty_cache()

In [40]:
# Run on a single batch of 5
train_dataloader = trainer.get_train_dataloader()

for batch in train_dataloader:
  # Check the batch structure
  print(f"input_ids shape {batch['input_ids'].shape}")
  print(f"attention_mask shape {batch['attention_mask'].shape}")
  print(f"labels shape {batch['labels'].shape}\n")
  break

input_ids shape torch.Size([5, 512])
attention_mask shape torch.Size([5, 512])
labels shape torch.Size([5, 512])



In [41]:
# If cuda:
tag_model_custom.train()  # Set model to training mode

# Move class_weights to the same device as the model
class_weights = class_weights.to(device)
input_ids = batch['input_ids'].to(device)
attention_mask = batch['attention_mask'].to(device)
labels = batch['labels'].to(device)
tag_model_custom = tag_model_custom.to(device)
loss_fct = loss_fct.to(device)

print(f"Model device: {tag_model_custom.device}")
print(f"Class weights device: {class_weights.device}")
print(f"Input IDs device: {input_ids.device}")
print(f"Attention mask device: {attention_mask.device}")
print(f"Labels device: {labels.device}")

# Ensure inputs are on the same device
outputs = tag_model_custom(
    input_ids=batch['input_ids'].to(device),
    attention_mask=batch['attention_mask'].to(device),
    labels=batch['labels'].to(device)
)
loss = outputs.loss
print(f"logits shape: {outputs.logits.shape}")
print(f"loss: {loss}")
loss.backward()

optimizer = torch.optim.AdamW(tag_model_custom.parameters(), lr=learning_rate)
optimizer.step()
optimizer.zero_grad()

Model device: cuda:0
Class weights device: cuda:0
Input IDs device: cuda:0
Attention mask device: cuda:0
Labels device: cuda:0
logits shape: torch.Size([5, 512, 66])
loss: 3.9450035095214844


In [ ]:
# If no cuda - uncomment and run
# tag_model_custom.train()  # Set model to training mode

# outputs = tag_model_custom(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
# loss = outputs.loss
# print(f"logits shape: {outputs.logits.shape}")
# print(f"loss: {loss}")
# loss.backward()

# optimizer = torch.optim.AdamW(tag_model_custom.parameters(), lr=learning_rate)
# optimizer.step()
# optimizer.zero_grad()

In [42]:
# Check functioning of trainer directly

batch = next(iter(train_dataloader))  # Get a single batch
outputs = trainer.training_step(trainer.model, batch)

print(outputs)  # Loss for the batch

tensor(3.2344, device='cuda:0')


In [43]:
# forward propagation, loss computation, backpropagation, and optimization all handled by Hugging Face Trainer
trainer.train()
trainer.push_to_hub(commit_message="NER Training")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,F1 Score
1,0.115400,0.106710,0.784747
2,0.072900,0.076942,0.844982
3,0.050000,0.067097,0.871083
4,0.032400,0.067407,0.883859
5,0.019900,0.065810,0.894153
6,0.019200,0.067797,0.893597
7,0.010900,0.067238,0.901166
8,0.007400,0.071975,0.900755
9,0.005400,0.072019,0.904086
10,0.005100,0.071876,0.904658


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



events.out.tfevents.1735899202.2508c7b74aec.359.0:   0%|          | 0.00/57.5k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HeathStar/distilbert-NER-Math-finetuned/commit/145f04d3a3a293da5a962edd9ae7f88b3f315811', commit_message='NER Training', commit_description='', oid='145f04d3a3a293da5a962edd9ae7f88b3f315811', pr_url=None, repo_url=RepoUrl('https://huggingface.co/HeathStar/distilbert-NER-Math-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='HeathStar/distilbert-NER-Math-finetuned'), pr_revision=None, pr_num=None)

In [44]:
%cd '/content/drive/MyDrive/Colab Notebooks/Math_Graph'
tag_model_custom.save_pretrained("math_ner_model")
tag_tokenizer.save_pretrained("math_ner_model")

/content/drive/MyDrive/Colab Notebooks/Math_Graph


('math_ner_model/tokenizer_config.json',
 'math_ner_model/special_tokens_map.json',
 'math_ner_model/vocab.txt',
 'math_ner_model/added_tokens.json',
 'math_ner_model/tokenizer.json')